In [ ]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

# Import custom modules
from lib.FramesDataset import FramesDataset
from lib import network

# Global variables
WARMUP = 4
T_STEPS = 45
FRAME_SIZE = 20
HIDDEN_UNITS = 100

PATHS = []
PATH = PATHS[0]

# Set device to use on network
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using", DEVICE)

# Load previously trained model (2000 epochs)
model = network.RecurrentTemporalPrediction.load(
    hidden_units = HIDDEN_UNITS,
    frame_size = FRAME_SIZE,
    warmup = WARMUP,
    path = PATH + '.pt'
)

# Load dataset
train_dataset = FramesDataset('./processed_dataset.pkl', 'train', WARMUP)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128)
print("Training dataset length:", len(train_dataset))

In [ ]:
# Plot loss history
with open(PATH + '.pickle', 'rb') as p:
    loss_history = pickle.load(p)
    
plt.loglog(loss_history);
plt.xlabel('Epoch')
plt.ylabel('Mean loss')
plt.show()

In [ ]:
# Produce spike-triggered average
n_rand_stimuli = 15000

stimuli = {}
for i in range(HIDDEN_UNITS):
    stimuli[i] = []

for i in range(n_rand_stimuli):
    if i % 1000 == 0:
        print('Trial %i' % i)
    
    noise_shape = (1, WARMUP+T_STEPS, FRAME_SIZE**2)
    noise = np.random.normal(loc=0, scale=1, size=noise_shape)
    noise = torch.Tensor(noise).to(DEVICE)
    
    _, hidden_state = model(noise)

    units = hidden_state[0, 0, :] # Discard first batch dimension
    for i, unit in enumerate(units):
        if unit > 0:
            stimuli[i].append(noise[0, -1, :].cpu().detach().numpy())

averaged_stimuli = {}
for i in range(HIDDEN_UNITS):
    stimuli_len = len(stimuli[i])
    if stimuli_len:
        averaged_stimuli[i] = np.stack(stimuli[i])
        averaged_stimuli[i] = np.reshape(np.mean(stimuli[i], 0), (FRAME_SIZE, FRAME_SIZE))
        
print('Finished averaging stimuli')

In [ ]:
# Plot spike triggered averages
count = 0
total_count = 10

offset = 50
offset_count = 0

for i, key in enumerate(averaged_stimuli):
    offset_count += 1
    if offset_count < offset:
        continue
        
    fig, ax = plt.subplots(dpi=25)
    plt.imshow(averaged_stimuli[key], cmap='gray')
    plt.title('Unit %i, mean of %i stimuli' % (i, stimuli_len))
    plt.show()


In [ ]:
# Plot weights of input to recurrent layer
def plot_input_weights (model, hidden_units, frame_size):
    offset = 0
    
    weights = model.rnn.weight_ih_l0.cpu().detach().numpy()    
    weights = weights[offset:hidden_units+offset, :]
    
    im = np.zeros((frame_size, frame_size*hidden_units))
    for i in range(hidden_units):
        hidden_unit_weights = weights[i]
        im[:, i*frame_size:(i+1)*frame_size] = np.reshape(hidden_unit_weights, (frame_size, frame_size))
    
    font_size = 5.5
    font_size_title = 6
    
    fig, ax = plt.subplots(dpi=200)
    im_plt = plt.imshow(im, extent=[0, hidden_units, 0, 1], cmap='gray')
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cb_plt = plt.colorbar(im_plt, cax=cax)
    cb_plt.ax.tick_params(labelsize=font_size)
    
    ax.set_xticks(np.arange(0, hidden_units+1))
    ax.set_xticklabels(np.arange(0, hidden_units+1), size=font_size-2)
    ax.set_xlabel('Hidden unit', size=font_size)
    
    ax.set_yticks(np.arange(0, 2))
    ax.set_yticklabels(np.arange(0, 2), size=font_size)    
    
    ax.grid(which='major', color='w', linestyle='-', linewidth=1)
    
plot_input_weights(model, hidden_units = 10, frame_size = FRAME_SIZE)

In [ ]:
# Visualise network predictions
x, y = train_dataset[0]
x = x.to(DEVICE)
y = y.to(DEVICE)
pred, _ = model(torch.unsqueeze(x, 0))
pred = torch.squeeze(pred, 0)

steps = 10

fig, axes = plt.subplots(1, steps, dpi=200)
for i in range(steps):
    im_y = y[WARMUP+i].view(20, 20).detach().cpu().numpy()
    axes[i].imshow(im_y, cmap='gray')
    axes[i].axis('off')
axes[0].set_title('Target', size=7)
plt.show()

fig, axes = plt.subplots(1, steps, dpi=200)
for i in range(steps):
    im_pred = pred[i].view(20, 20).detach().cpu().numpy()
    axes[i].imshow(im_pred, cmap='gray')
    axes[i].axis('off')
axes[0].set_title('Predicted', size=7)
plt.show()